In [ ]:
import datetime as datetime
from dateutil.relativedelta import relativedelta
import dateutil.rrule as rr
import importlib as ilib
import logging
import matplotlib
import matplotlib.pyplot as plt
import os
import pandas as pd
import pandas.tseries.offsets as o
import plotly.offline as plo
import plotly.graph_objs as go

import omega.configuration as oc
import omega.core.chain as occ
import omega.core.instrument as oci

import numpy as np
import json
plo.init_notebook_mode(connected=True)

In [ ]:
# Disable logging output
log = logging.getLogger()
log.setLevel(logging.ERROR)

In [ ]:
stem = 'FC'
# Get Chain
fc = occ.FutureChain(stem, oci.FutureType.Outright)
contracts = fc.initialize_contracts(occ.Status.All)
data = fc.initialize_data(days_back=240, partial=True)

length = 4 
df = fc.chain
# dff = fc.get_continuous()

dfs = []
previous = '1900-01-01'
for idx, row in fc.chain.iterrows():
    ct, last = row.values
    # Get current dataframe
    cdf = fc.data[ct]
    cdf = cdf[-91:-1] if idx == 0 else cdf[(cdf.index >= previous) & (cdf.index < last)]
    # Update previous
    previous = row['LastDate']
    dfs.append(cdf)
dff = pd.concat(dfs)
bluecurve = go.Scatter(x=dff.index, y=dff['Close'])

In [ ]:
# Plot the continuous curve
fig = go.Figure(data=[bluecurve])
plo.iplot(fig)

In [ ]:
# Red curves
def get_value(df, day):
    try:
        return df[df.index <= day].iloc[-1]['Close']
    except:
        return None
    
length = 6 # of contracts

# days = ['2017-03-23', '2017-04-23', '2017-05-23', '2017-06-23', '2017-07-23', '2017-08-23', '2017-09-23', '2017-10-23', '2017-11-23', '2017-12-23', '2018-01-23', '2018-02-23', '2018-03-23']
# get same date in each month since the beginning
days = [min(dff.index) + relativedelta(months=i) for i in range(len(pd.unique(dff.index.strftime('%Y-%m'))))]
days = [d.strftime('%Y-%m-%d') for d in days]
index = range(length)

redcurves = []
n = 20 # number of x axis ticks (days) between prices (y coordinates) for contracts in the chain at time t
for d in days:
    actives = fc.chain[fc.chain['LastDate'] > d]
    #print('Day: {} - Tickers: {}'.format(d, actives['Ticker'].values))
    #log.warning('')
    ts = [actives.iloc[idx]['Ticker'] if (idx+1 ) <= len(actives) else None for idx in index]
    prices = [get_value(fc.data[x], d) if x is not None else None for x in ts]
    xaxis = [pd.to_datetime(d) + pd.DateOffset(i*n) for i in range(len(prices))]
    redcurves.append((xaxis,prices))


In [ ]:
traces = []
for c in redcurves:
    trace = go.Scatter(
        y = c[1], # prices
        x = c[0], # tickvalues (dates) n ticks (days) apart from each other
        mode = 'lines+markers',
        line=dict(color='red'),
        marker = dict(size = 2),
        opacity = 0.5
    )
    traces.append(trace)
    
layout = dict(title='Forward Horizon')

fig = go.Figure(data=[bluecurve] + traces, layout = layout)

plo.iplot(fig)